In [31]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:200% !important; }</style>"))

In [32]:
import sys  
sys.path.insert(0, '/Users/tigrank/Tigran/programming/cad_parser/bin/')

In [45]:
import os, time, requests, datetime, contextlib, argparse
import regex as re
from cad_lib import isnotebook, ROOT_DIR

HTTP_ATTEMPTS = 1000
CNTY_SFFX     = 'jones'

if isnotebook():
    from tqdm.notebook import tqdm, trange
else:
    from tqdm import tqdm, trange

def generate_session_id(url='http://www.jonescad.org/search.aspx?clientid=jonescad'):
    request_res  = requests.get(url)
    assigned_url = request_res.url
    session_id   = assigned_url.split('/')[3]
    assert re.match('\(S\(.*\)\)', idd), "Wrong format of the session ID"
    
    return session_id

In [57]:
if __name__ == '__main__':

    if isnotebook():
        fetch_properties, fetch_owners = True, True
    else:
        parser = argparse.ArgumentParser(description='What to fetch')
        parser.add_argument('--properties', type=bool, help='fetch properties', required=False, default=False)
        parser.add_argument('--owners', type=bool, help='fetch owners', required=False, default=False)
        args = parser.parse_args()
        fetch_properties, fetch_owners = args.properties, args.owners
    
    data_dir = f'{ROOT_DIR}/data/data_{CNTY_SFFX}'
    os.makedirs(data_dir, exist_ok=True)
    
    session_id = generate_session_id()
    
    begin_id, end_id = 15638, 15639 # change later
    
    for prop_id in trange(begin_id, end_id):
        url   = f'http://www.jonescad.org/{session_id}/rgeneral.aspx?ID={prop_id}&seq=1'
        fname = f'{data_dir}/prop_{prop_id}.html'
        
        # Handling stale sessions
        for trial in range(HTTP_ATTEMPTS):
            response = requests.get(url)
            
            if 'Welcome to the P&amp;A Website!' not in response.text:
                break
            
            if trial==HTTP_ATTEMPTS-1:
                raise Exception(f'Connection timeout at prop_id={prop_id}')
            
            time.sleep(1)
            session_id = generate_session_id()
            
        if fr'value="{prop_id}"' not in response.text:
            with contextlib.suppress(FileNotFoundError): # delete file, if property was removed from the county website
                os.remove(fname)
        else:
            with open(fname, 'wb') as f:
                f.write(response.content)